In [1]:
import random
import numpy as np

In [2]:
Q = 2653433339 # < 32 bits -- NumPy is weird when 64bit overflow happens (no arbitrary precision)

In [3]:
BASE = 2
PRECISION_INTEGRAL   = 5
PRECISION_FRACTIONAL = 10

assert BASE**(2*PRECISION_INTEGRAL + 2*PRECISION_FRACTIONAL) < Q

def encode(rationals):
    return (rationals * BASE**PRECISION_FRACTIONAL).astype(int) % Q

def decode(elements):
    map_negative_range = np.vectorize(lambda element: element if element <= Q/2 else element - Q)
    return map_negative_range(elements) / BASE**PRECISION_FRACTIONAL

In [4]:
class PublicTensor:
    
    def __init__(self, values):
        self.values = values
    
    def load(values, apply_encoding=True):
        if apply_encoding:
            values = encode(values)
        return PublicTensor(values)
    
    def truncate(self, amount=PRECISION_FRACTIONAL):
        truncate = np.vectorize(lambda  x: x // BASE**amount)
        return PublicTensor(truncate(self.values))
    
    def __repr__(self):
        return "PublicTensor(%s)" % decode(self.values)
        
    def __add__(x, y):
        if isinstance(y, PublicTensor):
            values = (x.values + y.values) % Q
            return PublicTensor(values)
        elif isinstance(y, PrivateTensor):
            shares0 = (x.values + y.shares0) % Q
            shares1 =             y.shares1
            return PrivateTensor(shares0, shares1)
    
    def __sub__(x, y):
        if isinstance(y, PublicTensor):
            values = (x.values - y.values) % Q
            return PublicTensor(values)
        elif isinstance(y, PrivateTensor):
            shares0 = (x.values - y.shares0) % Q
            shares1 =             y.shares1
            return PrivateTensor(shares0, shares1)
    
    def __mul__(x, y):
        if isinstance(y, PublicTensor):
            values = (x.values * y.values) % Q
            return PublicTensor(values)
        elif isinstance(y, PrivateTensor):
            shares0 = (x.values * y.shares0) % Q
            shares1 = (x.values * y.shares1) % Q
            return PrivateTensor(shares0, shares1)
    
    def dot(x, y):
        if isinstance(y, PublicTensor):
            return PublicTensor(x.values.dot(y.values))
        elif isinstance(y, PrivateTensor):
            shares0 = x.values.dot(y.shares0) % Q
            shares1 = x.values.dot(y.shares1) % Q
            return PrivateTensor(shares0, shares1)
        
    def transpose(self):
        return PublicTensor(self.values.T)
    
    def sum0(self):
        values = self.values.sum(axis=0, keepdims=True)
        return PublicTensor(values)

In [5]:
x = PublicTensor.load(np.array([1,2,3,4]).reshape(4,1))
y = PublicTensor.load(np.array([5,6,7,8]).reshape(4,1))
z = x + y; print(z)
z = (x * y).truncate(); print(z)
z = x.dot(y.transpose()).truncate(); print(z)
z = z.sum0(); print(z)

PublicTensor([[  6.]
 [  8.]
 [ 10.]
 [ 12.]])
PublicTensor([[  5.]
 [ 12.]
 [ 21.]
 [ 32.]])
PublicTensor([[  5.   6.   7.   8.]
 [ 10.  12.  14.  16.]
 [ 15.  18.  21.  24.]
 [ 20.  24.  28.  32.]])
PublicTensor([[ 50.  60.  70.  80.]])


In [6]:
def generate_mul_triple(shape):
    a = np.random.randint(Q, size=shape)
    b = np.random.randint(Q, size=shape)
    ab = (a * b) % Q
    return PrivateTensor.load(a, False), \
           PrivateTensor.load(b, False), \
           PrivateTensor.load(ab, False)

In [7]:
def generate_dot_triple(m, n, o):
    a = np.random.randint(Q, size=(m, n))
    b = np.random.randint(Q, size=(n, o))
    ab = np.dot(a, b)
    return PrivateTensor.load(a, False), \
           PrivateTensor.load(b, False), \
           PrivateTensor.load(ab, False)

In [8]:
class PrivateTensor:
    
    def __init__(self, shares0, shares1):
        self.shares0 = shares0
        self.shares1 = shares1
    
    def load(values, apply_encoding=True):
        if apply_encoding:
            values = encode(values)
        shares0 = np.random.randint(Q, size=values.shape)
        shares1 = (values - shares0) % Q
        return PrivateTensor(shares0, shares1)

    def reveal(self):
        values = (self.shares0 + self.shares1) % Q
        return PublicTensor(values)
    
    def truncate(self, amount=PRECISION_FRACTIONAL):
        truncate0 = np.vectorize(lambda x0: x0 // BASE**amount)
        truncate1 = np.vectorize(lambda x1: Q - ((Q - x1) // BASE**amount))
        return PrivateTensor(truncate0(self.shares0), truncate1(self.shares1))
    
    def __repr__(self):
        return "PrivateTensor(%s)" % decode(self.reveal().values)
    
    def __add__(x, y):
        if isinstance(y, PrivateTensor):
            shares0 = (x.shares0 + y.shares0) % Q
            shares1 = (x.shares1 + y.shares1) % Q
            return PrivateTensor(shares0, shares1)
        elif isinstance(y, PublicTensor):
            shares0 = (x.shares0 + y.values) % Q
            shares1 =  x.shares1
            return PrivateTensor(shares0, shares1)
    
    def __sub__(x, y):
        if isinstance(y, PrivateTensor):
            shares0 = (x.shares0 - y.shares0) % Q
            shares1 = (x.shares1 - y.shares1) % Q
            return PrivateTensor(shares0, shares1)
        elif isinstance(y, PublicTensor):
            shares0 = (x.shares0 - y.values) % Q
            shares1 =  x.shares1
            return PrivateTensor(shares0, shares1)
    
    def __mul__(x, y):
        if isinstance(y, PrivateTensor):
            assert x.shares0.shape == x.shares1.shape == y.shares0.shape == y.shares1.shape
            a, b, ab = generate_mul_triple(x.shares0.shape)
            alpha = (x - a).reveal()
            beta  = (y - b).reveal()
            return alpha*beta + alpha*b + a*beta + ab
        elif isinstance(y, PublicTensor):
            shares0 = (x.shares0 * y.values) % Q
            shares1 = (x.shares1 * y.values) % Q
            return PrivateTensor(shares0, shares1)
        
    def dot(x, y):
        if isinstance(y, PrivateTensor):
            assert x.shares0.shape == x.shares1.shape 
            assert y.shares0.shape == y.shares1.shape
            m = x.shares0.shape[0]
            n = x.shares0.shape[1]
            o = y.shares0.shape[1]
            assert n == y.shares0.shape[0]
            a, b, ab = generate_dot_triple(m, n, o)
            alpha = (x - a).reveal()
            beta  = (y - b).reveal()
            return alpha.dot(beta) + alpha.dot(b) + a.dot(beta) + ab
        elif isinstance(y, PublicTensor):
            shares0 = x.shares0.dot(y.values) % Q
            shares1 = x.shares1.dot(y.values) % Q
            return PrivateTensor(shares0, shares1)
        
    def transpose(self):
        return PrivateTensor(self.shares0.T, self.shares1.T)
    
    def sum0(self):
        shares0 = self.shares0.sum(axis=0, keepdims=True) % Q
        shares1 = self.shares1.sum(axis=0, keepdims=True) % Q
        return PrivateTensor(shares0, shares1)

In [9]:
x = PrivateTensor.load(np.array([1,2,3,4]).reshape(4,1))
y = PrivateTensor.load(np.array([5,6,7,8]).reshape(4,1))
z = x + y; print(z)
z = (x * y).truncate(); print(z)
z = x.dot(y.transpose()).truncate(); print(z)
z = z.sum0(); print(z)

PrivateTensor([[  6.]
 [  8.]
 [ 10.]
 [ 12.]])
PrivateTensor([[  5.]
 [ 12.]
 [ 21.]
 [ 32.]])
PrivateTensor([[  5.   6.   7.   8.]
 [ 10.  12.  14.  16.]
 [ 15.  18.  21.  24.]
 [ 20.  24.  28.  32.]])
PrivateTensor([[ 50.  60.  70.  80.]])


In [10]:
x = np.array(np.arange(16)).reshape(4,4)
y = x.sum(axis=0, keepdims=True)
print(x, y)

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]] [[24 28 32 36]]
